### Importações

In [ ]:
%%capture
!pip install transformers wptools

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import transformers
import json
import wptools
import base64
import sys
import re
import datetime
import tabulate
import locale

### Configurações iniciais

In [ ]:
_ = datetime.datetime.now().astimezone(datetime.timezone(datetime.timedelta(hours=-3))).strftime('%d-%m_%H-%M')
original_spreadsheet_path = '/content/drive/MyDrive/Unisinos/2021_1/60583 - Projeto Final II/result.xlsx'
result_spreadsheet_path = f'/content/drive/MyDrive/Unisinos/2021_1/60583 - Projeto Final II/outputs/output_{_}.xlsx'
# result_spreadsheet_path = original_spreadsheet_path ####################################################################
writer = pd.ExcelWriter(result_spreadsheet_path)
df_original = pd.read_excel(original_spreadsheet_path, sheet_name='original')
df_original.to_excel(writer, sheet_name='original', index=False, freeze_panes=(1,0))

In [ ]:
# Maintain the data
df_filtered = pd.read_excel(original_spreadsheet_path, sheet_name='filtered')
df_filtered.to_excel(writer, sheet_name='filtered', index=False, freeze_panes=(1,0))
df_sorted = pd.read_excel(original_spreadsheet_path, sheet_name='sorted')
df_sorted.to_excel(writer, sheet_name='sorted', index=False, freeze_panes=(1,0))
df_nltk = pd.read_excel(original_spreadsheet_path, sheet_name='NLTK')
df_nltk.to_excel(writer, sheet_name='NLTK', index=False, freeze_panes=(1,0))
df_bert = pd.read_excel(original_spreadsheet_path, sheet_name='XLM-RoBERTa')
df_bert.to_excel(writer, sheet_name='XLM-RoBERTa', index=False, freeze_panes=(1,0))
df_categories_manual = pd.read_excel(original_spreadsheet_path, sheet_name='categories_manual')
df_categories_manual.to_excel(writer, sheet_name='categories_manual', index=False, freeze_panes=(1,0))
df_categories_clean = pd.read_excel(original_spreadsheet_path, sheet_name='categories_clean')
df_categories_clean.to_excel(writer, sheet_name='categories_clean', index=False, freeze_panes=(1,0))
df_categories = pd.read_excel(original_spreadsheet_path, sheet_name='categories')
df_categories.to_excel(writer, sheet_name='categories', index=False, freeze_panes=(1,0))
df_leaked_data = pd.read_excel(original_spreadsheet_path, sheet_name='leaked_data')
df_leaked_data.to_excel(writer, sheet_name='leaked_data', index=False, freeze_panes=(1,0))
df_citizen = pd.read_excel(original_spreadsheet_path, sheet_name='citizen')
df_citizen.to_excel(writer, sheet_name='citizen', index=False, freeze_panes=(1,0))
df_categories_2 = pd.read_excel(original_spreadsheet_path, sheet_name='categories_2')
df_categories_2.to_excel(writer, sheet_name='categories_2', index=False, freeze_panes=(1,0))

### Limpeza das informações

In [ ]:
def _filtering_function(row):
  title = row['title'].lower()
  description = row['description'].lower()
  for _ in ["database", "leak", "leack"]:
    if (_ in title): # or (_ in description):
      for _ in ["★", "system requirements", "hacker", "warranty", "collection", "pack", "course"]:
        if (_ in title) or (_ in description):
          return False
      return True
  return False

df_filter = df_original.apply(_filtering_function, axis='columns')
df_filtered = df_original[df_filter]
df_filtered.to_excel(writer, sheet_name='filtered', index=False, freeze_panes=(1,0))


In [ ]:
df_sorted = df_filtered.copy()
df_sorted['sort_title'] = df_sorted['title'].apply(lambda x: x.lower())
df_sorted.rename(columns={'title': 'original_title', 'sort_title': 'title'}, inplace=True)
_c = df_sorted.columns.to_list()
df_sorted = df_sorted[[_c[0]] + [_c[-1]] + _c[1:-1]]

df_sorted.replace({
  'title': {
    r'^(?:\d{4} - )?(?:\s+?)?\([^\)]*?\)(?:\s+?)?': '', 
    r'\s+': ' ',
    r'^\s': '',
    r'\s$': '',
    r'\/': ' ',
    r',': '',
    r'(?i)^(?:full)\s+(?:and complete)?\s*': ''
  },
  'description':{
      r"\r\n": "\n",
      r"(?i)\s*(?:Why Buy from us|We promise|Refund Policy|Regards)[\s\S]*$": ""
  }
}, regex=True, inplace=True)

df_sorted.sort_values(['title', 'seller'], ignore_index=True, inplace=True)
df_sorted.to_excel(writer, sheet_name='sorted', index=False, freeze_panes=(1,0))

### NLTK

In [ ]:
nltk.download(['punkt', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words']);

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
def get_continuous_chunks(text, labels=['ORGANIZATION', 'PERSON', 'GPE']):
  def filter_words(p):
    r = []
    for i in p:
      l = i.lower()
      if l in ['']:
        continue
      i = re.sub(r'(?i)(?:hacked|leacked|leack|leaked|leak|database|business|full)', '', i)
      i = re.sub(r'(?i)\s+', ' ', i).strip()
      if not i:
        continue
      r.append(i)
    return r
  chunked = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
  prev = None
  continuous_chunk = []
  current_chunk = []
  for subtree in chunked:
    if type(subtree) == nltk.tree.Tree and subtree.label() in labels:
      current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
    if current_chunk:
      named_entity = " ".join(current_chunk)
      if named_entity not in continuous_chunk:
        continuous_chunk.append(named_entity)
        current_chunk = []
    else:
      continue
  return filter_words(continuous_chunk)

In [ ]:
df_nltk = df_sorted.copy()
df_nltk['entities'] = df_nltk['original_title'].apply(lambda x: json.dumps(get_continuous_chunks(x)))
df_nltk.to_excel(writer, sheet_name='NLTK', index=False, freeze_panes=(1,0))

### BERT


In [ ]:
nlp = transformers.pipeline('ner',
    model="xlm-roberta-large-finetuned-conll03-english",
    tokenizer="xlm-roberta-large-finetuned-conll03-english"
)

In [ ]:
def read_organizations(response):
  def fix_domains(entity_list):
    r = []
    for i in entity_list:
      l = i.lower()
      if l in ['database', 'forum', 'business', 'consumer', 'leacked', 'leack', 'leaked', 'leak']:
        continue
      if len(l) <= 3:
        continue
      for t in  ["com", "net", "org", "tv", "io", "ly", "cn", "co", "za", "kr", "in", "club", "fm"]:
        cpos = len(t) * -1
        if l[cpos:].lower() == t and l[(cpos - 1):cpos].lower() != '.':
          i = (i[:cpos] + '.' + t).lower()
      r.append(i)
    return r
  
  organizations = []
  current_token = ''
  for token in response:
    if 'ORG' not in token['entity']:
      continue
    is_initial_string = token['word'][0] == '\u2581'
    if is_initial_string:
      if current_token:
        organizations.append(current_token)
      current_token = token['word'][1:]
    else:
      current_token += token['word']
  
  return fix_domains(list(filter(None, organizations + [current_token])))

In [ ]:
df_bert = df_sorted.copy()
df_bert['entities'] = df_bert['original_title'].apply(lambda x: json.dumps(read_organizations(nlp(x, grouped_entities=True))))
df_bert.to_excel(writer, sheet_name='XLM-RoBERTa2', index=False, freeze_panes=(1,0))

### Categorização das organizações

In [ ]:
_cache = {}
def get_wikipedia_page_details(item):
  print(item)
  if item in _cache:
    return _cache[item]
  try:
    _page = wptools.page(item, silent=True).get().data
    if (not _page) or (not _page['wikidata']):
      _cache[item] = None
      return None
    _result = {'title': _page['title']}
    for i in ['industry (P452)']:
      if i in _page['wikidata']:
        _result[i] = _page['wikidata'][i]
    if _result:
      _cache[item] = _result
      return _result
    _cache[item] = None
    return None
  except (ValueError, LookupError) as e:
    _cache[item] = None
    return None
def filter_entities(entity):
  if not entity:
    return False
  if entity.lower() in ['database', 'forum', 'business', 'consumer']:
    return False
  return True

def join_entities_and_get_categories(row, modes=['nltk', 'bert']):
  for mode in modes:
    raw_entities = row['entities_' + mode]
    if type(raw_entities) != 'list':
      raw_entities = json.loads(raw_entities)
    filtered_entities = list(filter(filter_entities, raw_entities))
    if not filtered_entities:
      continue
    original_text = ' '.join(filtered_entities)
    original_text = re.sub(r'(?i)(database|hacked|leaked|leacked|leak|leack)\s*', '', original_text)
    if len(original_text) <= 3:
      continue
    search_texts = [original_text]
    if original_text[-3:].lower() == 'com' and original_text[-4:-3].lower() != '.':
      search_texts.append(original_text[:-3] + '.com')
    if original_text[-3:].lower() == 'net' and original_text[-4:-3].lower() != '.':
      search_texts.append(original_text[:-3] + '.net')
    for text in search_texts:
      result = get_wikipedia_page_details(text)
      if result:
        return json.dumps(result)
  return ''

In [ ]:
sys.exit(1)
if False:
  df_categories_manual = df_sorted.copy()
  df_categories_manual['entities_nltk'] = df_nltk['entities']
  df_categories_manual['entities_bert'] = df_bert['entities']
  df_categories_manual['categories'] = df_categories_manual.apply(join_entities_and_get_categories, axis='columns')

  _ = lambda x: ''
  df_categories_manual = df_categories_manual.assign(org_name=_, site_url=_, delete=_, gics_industry=_,	gics_category=_, data_category=_, additional_data=_)

  df_categories_manual.to_excel(writer, sheet_name='categories_manual', index=False, freeze_panes=(1,0))

In [ ]:
df_temp = df_categories_manual.copy()
df_filter = df_temp.apply(lambda x: pd.isna(x['delete']), axis='columns')
df_categories_clean = df_temp[df_filter]
df_categories_clean = df_categories_clean.drop(columns=['delete'])
df_categories_clean.reset_index(drop=True, inplace=True)
df_categories_clean.to_excel(writer, sheet_name='categories_clean', index=False, freeze_panes=(1,0))

### Identificação do número de registros

In [ ]:
rules = [
  {
    "regex": re.compile(r'(?i)(?P<full>(?P<records>(?:\d+[,\. ]?)?(?:\d+[,\. ]?)?\d+)\+?(?:\r\n|\s)?(?P<unit>government|record|records|entries|citizens|contacts|voters|accounts|emails|users|email:pass|lines|\(plaintext\)))'),
    "unit": 1,
    "fields": ['original_title', 'description'],
  },
  {
    "regex": re.compile(r'(?i)(?P<full>(entry|count|records|total|lines)\s?:?(?:\r\n|\s)?(?P<records>(?:\d+[,\. ]?)?(?:\d+[,\. ]?)?\d+)\+?(?:\s|\r\n)?)'),
    "unit": 1,
    "fields": ['description'],
  },
  {
    "regex": re.compile(r'(?i)(?P<full>(?P<records>(?:\d+[,\. ]?)?(?:\d+[,\. ]?)?\d+))$'),
    "unit": 1,
    "fields": ['title'],
  },
  {
    "regex": re.compile(r'(?i)(?P<full>(?P<records>(?:\d+[,\. ]?)?(?:\d+[,\. ]?)?\d+)\+?(?:\r\n|\s)?(?P<unit>k)(?:\r\n|\s|$))'),
    "unit": 1000,
    "fields": ['original_title', 'description'],
  },
  {
    "regex": re.compile(r'(?i)(?P<full>(?P<records>(?:\d+[,\. ]?)?(?:\d+[,\. ]?)?\d+)\+?(?:\r\n|\s)?(?P<unit>million|m)(?:\+|\r\n|\s|$))'),
    "unit": 1000000,
    "fields": ['original_title', 'description'],
  },
]

def count_records(row):
  for rule in rules:
    regex = rule["regex"]
    unit = rule["unit"]
    for field in rule["fields"]:
      text = row[field].strip()
      match = regex.search(text)

      if not match:
        continue

      records = match.group('records')
      records = records.strip('\n\r ').replace(' ', '')

      if records in ['2019', '2020', '2021']:
        continue

      if unit == 1:
        records = records.replace(',', '').replace('.', '')
        if float(records) < 100:
          continue
      else:
        records = records.replace(',', '.')
        dot_count = records.count('.')
        records = records.replace('.', '', dot_count - 1)
      return float(records) * unit
  return 0

df_categories = df_categories_clean.copy()

df_categories['records'] = df_categories.apply(count_records, axis='columns')

df_categories.to_excel(writer, sheet_name='categories', index=False, freeze_panes=(1,0))

### Identificação dos dados vazados

In [ ]:
_regex = {
    'email':               [re.compile(r"(?mi)(email|e\s?mail\saddress|e\saddress)")],
    'username':            [re.compile(r"(?mi)(username)")],
    'name':                [re.compile(r"(?mi)(\bnames?\b)")],
    'password':            [re.compile(r"(?mi)(pass|password)s?")],
    'salt':                [re.compile(r"(?mi)(salt)s?")],
    'password_hint':       [re.compile(r"(?mi)(password\shint|security\squestions)")],
    '2fa':                 [re.compile(r"(?mi)(2fa)")],
    'plaintext':           [re.compile(r"(?mi)(plaintext|plain)")],
    'hashed':              [re.compile(r"(?mi)(hash|md5|sha\d|bcrypt|ipb)")],
    'phone':               [re.compile(r"(?mi)((?<!i)phone)")],
    'ip':                  [re.compile(r"(?mi)(\bip\b|ipaddress)")],
    'physical_address':    [re.compile(r"(?mi)(physical|location)s?")],
    'geolocation':         [re.compile(r"(?mi)(geographic\slocations)s?")],
    'payment':             [re.compile(r"(?mi)(payment)\s(histor(y|ies)|methods?)")],
    'gender':              [re.compile(r"(?mi)(gender)s?")],
    'dob':                 [re.compile(r"(?mi)(dob|birth)")],
    'device_info':         [re.compile(r"(?mi)(device\sinformation|device\sid|ipad|iphone|ipod)s?")],
    'site_activity':       [re.compile(r"(?mi)(forum|site)\s(activity|usage)")],
    'registration_date':   [re.compile(r"(?mi)(registration\sdate)")],
    'voter_ids':           [re.compile(r"(?mi)(voter\sid)s?")],
    'citizen_status':      [re.compile(r"(?mi)(citizen\sstatus)")],
    'contacts':            [re.compile(r"(?mi)(contacts)")],
    'sexual_orientations': [re.compile(r"(?mi)(sexual\sorientation)s?")],
    'credit_card':         [re.compile(r"(?mi)(credit\scard)s?")],
    'spoken_languages':    [re.compile(r"(?mi)(spoken\slanguage)s?")],
    'survey_results':      [re.compile(r"(?mi)(survey\sresults)")],
    'mesenger':            [re.compile(r"(?mi)(instant\sidentit(?:y|ies)|instant\smessenger\sidentit(?:y|ies)|private\smessages?)")],
    'balances':            [re.compile(r"(?mi)(account\sbalance)s?")],
    'employers':           [re.compile(r"(?mi)(employers)")],
    'passport':            [re.compile(r"(?mi)(passport)s?")],
}

df_leaked_data = df_categories.copy()

def _has(row, list_of_regex):
  for i in list_of_regex:
    if i.search(row['description']):
      return 1
  return 0

for i in _regex:
  df_leaked_data[i] = df_leaked_data.apply(lambda x: _has(x, _regex[i]), axis='columns')

df_leaked_data.to_excel(writer, sheet_name='leaked_data', index=False, freeze_panes=(1,0))

### Citizen

In [ ]:
sys.exit(1)
if False:
  temp_df = df_leaked_data.copy()
  df_citizen = temp_df.loc[df_leaked_data.data_category == 'citizen']
  _ = lambda x: ''
  df_citizen = df_citizen.assign(country=_, state=_)
  df_citizen.to_excel(writer, sheet_name='citizen', index=False, freeze_panes=(1,0))

### Plot

In [ ]:
def format_table(text):
  return re.sub(r' {2,}', ' ', text.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline'))

def real_br_money_mask(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')
  
filter_all_df = (~df_leaked_data['data_category'].isin(['?', 'unknow'])) & (df_leaked_data['gics_industry'] != 'unknow')
all_df = df_leaked_data[filter_all_df]
citizen_df = df_citizen
filter_org_df = ~all_df['data_category'].isin(['citizen', 'combo', 'phone', 'email'])
org_df = all_df[filter_org_df]
accuracy_df = df_categories_2[filter_all_df][filter_org_df]
all_df.to_excel(writer, sheet_name='final_all_df', index=False, freeze_panes=(1,0))
citizen_df.to_excel(writer, sheet_name='final_citizen_df', index=False, freeze_panes=(1,0))
org_df.to_excel(writer, sheet_name='final_org_df', index=False, freeze_panes=(1,0))
accuracy_df.to_excel(writer, sheet_name='final_accuracy_df', index=False, freeze_panes=(1,0))

### Vendedores

In [ ]:
# Vendedores
temp_df = []
for seller in org_df['seller'].unique():
  row = []
  row.append(seller)
  d = org_df[org_df['seller'] == seller]
  row.append(len(d))
  row.append(len(d['site_slug'].unique()))
  row.append(len(d['gics_industry'].unique()))
  row.append(d['views'].sum())
  # if len(d) == 0:
  #   continue
  # d = d.describe(percentiles=[.5])
  # row.append(int(d['count']))
  # row.append(int(d['min']))
  # row.append(int(d['50%']))
  # row.append(int(d['max']))
  temp_df.append(row)  

#temp_df = []
df_sellers = pd.DataFrame(temp_df, columns=['Vendedor', 'Qtde. anúncios', 'Qtde. marketplaces', 'Qtde. setores', 'Visualizações'])
df_sellers.sort_values(by=df_sellers.columns[1], inplace=True, ignore_index=True, ascending=False)
format_table(df_sellers.to_latex(
  index=False, 
  caption='Número de anúncios por vendedor.', 
  label='tab:top10sellers', 
  column_format="l|c|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_sellers.columns.values],
  escape=False
))
#display(df_sellers)


### NLTK vs XLM-RoBERTa

In [ ]:
# Assertividade do NLTK e do BERT
temp_df = []

tp1 = len(accuracy_df[(accuracy_df['avaliacao_nltk'] == 'TP') & (accuracy_df['avaliacao_bert'] != 'TP')])
fp1 = len(accuracy_df[(accuracy_df['avaliacao_nltk'] == 'FP') & (accuracy_df['avaliacao_bert'] != 'FP')])
tn1 = len(accuracy_df[(accuracy_df['avaliacao_nltk'] == 'TN') & (accuracy_df['avaliacao_bert'] != 'TN')])
fn1 = len(accuracy_df[(accuracy_df['avaliacao_nltk'] == 'FN') & (accuracy_df['avaliacao_bert'] != 'FN')])

tp2 = len(accuracy_df[(accuracy_df['avaliacao_bert'] == 'TP') & (accuracy_df['avaliacao_nltk'] != 'TP')])
fp2 = len(accuracy_df[(accuracy_df['avaliacao_bert'] == 'FP') & (accuracy_df['avaliacao_nltk'] != 'FP')])
tn2 = len(accuracy_df[(accuracy_df['avaliacao_bert'] == 'TN') & (accuracy_df['avaliacao_nltk'] != 'TN')])
fn2 = len(accuracy_df[(accuracy_df['avaliacao_bert'] == 'FN') & (accuracy_df['avaliacao_nltk'] != 'FN')])

tp = tp1 + tp2
fp = max(fp1, fp2)
tn = tn1 + tn2
fn = max(fn1, fn2)

print(f'tp = {tp}\nfp = {fp}\ntn = {tn}\nfn = {fn}\n\n')
precision = (tp/(tp + fp))
recall =  (tp/(tp + fn))
f1 = ((2 * precision * recall)/(precision + recall))
accuracy = ((tn + tp)/(tn + fp + fn + tp))
print(accuracy, precision, recall, f1)

### Indústria

In [ ]:
# Setores da indústria mais afetados
temp_series = org_df.value_counts(['gics_industry']).head(10)
total = len(org_df)
temp_df = []
for industry, count in temp_series.iteritems():
  row = []
  row.append(industry[0].replace('&', '\\&'))
  row.append(count)
  row.append(real_br_money_mask((count * 100) / total))
  row.append(real_br_money_mask(org_df[org_df['gics_industry'] == industry[0]]['views'].sum())[:-3])
  d = org_df[(org_df['gics_industry'] == industry[0]) & (org_df['records'] != 0)]['records']
  if len(d) == 0:
    i = "-"
    row.append(i)
  else:
    i = d.sum() / len(d) / 1000000
    row.append(real_br_money_mask(i))

  temp_df.append(row)

df_per_industry = pd.DataFrame(temp_df, columns=['Setor da indústria', 'Número de registros', 'Porcentagem do total (\%)', 'Visualizações', 'Média registros'])
df_per_industry.sort_values(by=df_per_industry.columns[1], ascending=False, inplace=True)
df_per_industry.reset_index(drop=True)
format_table(df_per_industry.to_latex(
  index=False, 
  caption='Setores da indústria com mais vazamentos', 
  label='tab:top10setores', 
  column_format="l|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_per_industry.columns.values],
  escape=False
))
#df_per_industry

In [ ]:
# Variação de registros por categoria
temp_df = []
for category in org_df['gics_industry'].unique():
  row = []
  row.append(category.replace('&', '\\&'))
  d = org_df[(org_df['gics_industry'] == category) & (org_df['records'] != 0)]['records']
  if len(d) == 0:
    continue
  d = d.describe(percentiles=[.5])
  row.append(int(d['count']))
  row.append(int(d['min']))
  row.append(int(d['50%']))
  row.append(int(d['max']))
  temp_df.append(row)  

df_records_per_idustry = pd.DataFrame(temp_df, columns=['Indústria', 'Qtde. anúncios', 'Mínimo', 'Mediana', 'Máximo'])
df_records_per_idustry.sort_values(by=df_records_per_idustry.columns[4], inplace=True, ignore_index=True, ascending=False)
df_records_per_idustry.drop(columns=[df_records_per_idustry.columns[1]], inplace=True)
format_table(df_records_per_idustry.head(10).to_latex(
  index=False, 
  caption='Número de registros por vazamentos e indústria', 
  label='tab:top10recordsperindustry', 
  column_format="l|c|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_records_per_idustry.columns.values],
  escape=False
))
display(df_records_per_idustry)


In [ ]:
# Views por categoria
temp_df = []
for category in org_df['gics_industry'].unique():
  row = []
  row.append(category.replace('&', '\\&'))
  d = org_df[(org_df['gics_industry'] == category) & (org_df['views'] != 0)]['views']
  if len(d) == 0:
    continue
  sum = d.sum()
  d = d.describe(percentiles=[.5])
  row.append(int(d['count']))
  row.append(int(d['min']))
  row.append(int(d['50%']))
  row.append(int(d['max']))
  row.append(int(sum))
  temp_df.append(row)  

df_views_per_idustry = pd.DataFrame(temp_df, columns=['Indústria', 'Qtde. anúncios', 'Mínimo', 'Mediana', 'Máximo', 'Soma'])
df_views_per_idustry.sort_values(by=df_views_per_idustry.columns[3], inplace=True, ignore_index=True, ascending=False)
df_views_per_idustry.drop(columns=[df_views_per_idustry.columns[1]], inplace=True)
format_table(df_views_per_idustry.head(10).to_latex(
  index=False, 
  caption='Número de views por anúncio e indústria', 
  label='tab:top10viewsperindustry', 
  column_format="l|c|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_views_per_idustry.columns.values],
  escape=False
))
display(df_views_per_idustry.head(10))


In [ ]:
# Variação de preço por categoria
temp_df = []
for category in org_df['gics_industry'].unique():
  row = []
  row.append(category.replace('&', '\\&'))
  z = org_df[org_df['gics_industry'] == category]
  price_per_record = (org_df[(org_df['gics_industry'] == category) & (org_df['records'] != 0)]['price'].sum() / org_df[(org_df['gics_industry'] == category) & (org_df['records'] != 0)]['records'].sum()) *1000000
  d = z['price'].describe(percentiles=[.5,.75, .85, .90, .95])
  row.append(int(d['count']))
  row.append(real_br_money_mask(d['min']))
  row.append(real_br_money_mask(d['50%']))
  row.append(real_br_money_mask(d['80%']))
  row.append(real_br_money_mask(d['max']))
  row.append(real_br_money_mask(price_per_record))
  temp_df.append(row)  

df_price_per_idustry = pd.DataFrame(temp_df, columns=['Indústria', 'Qtde. anúncios', 'Mínimo', 'Mediana', '75\%', 'Máximo', 'Valor por registro'])
df_price_per_idustry.sort_values(by=df_price_per_idustry.columns[1], inplace=True, ignore_index=True, ascending=False)
df_price_per_idustry.drop(columns=[df_price_per_idustry.columns[1]], inplace=True)
format_table(df_price_per_idustry.head(10).to_latex(
  index=False, 
  caption='Valores dos vazamentos por indústria (Valores em USD)', 
  label='tab:top10priceperindustry', 
  column_format="l|r|r|r|r|r|r|r",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_price_per_idustry.columns.values],
  escape=False
))
display(df_price_per_idustry.head(10))


### Dados

In [ ]:
# Dados mais vazados
_regex={'email':[re.compile('(?mi)(email|e\\s?mail\\saddress|e\\saddress)')],'username':[re.compile('(?mi)(username)')],'name':[re.compile('(?mi)(\\bnames?\\b)')],'password':[re.compile('(?mi)(pass|password)s?')],'salt':[re.compile('(?mi)(salt)s?')],'password_hint':[re.compile('(?mi)(password\\shint|security\\squestions)')],'2fa':[re.compile('(?mi)(2fa)')],'plaintext':[re.compile('(?mi)(plaintext|plain)')],'hashed':[re.compile('(?mi)(hash|md5|sha\\d|bcrypt|ipb)')],'phone':[re.compile('(?mi)((?<!i)phone)')],'ip':[re.compile('(?mi)(\\bip\\b|ipaddress)')],'physical_address':[re.compile('(?mi)(physical|location)s?')],'geolocation':[re.compile('(?mi)(geographic\\slocations)s?')],'payment':[re.compile('(?mi)(payment)\\s(histor(y|ies)|methods?)')],'gender':[re.compile('(?mi)(gender)s?')],'dob':[re.compile('(?mi)(dob|birth)')],'device_info':[re.compile('(?mi)(device\\sinformation|device\\sid|ipad|iphone|ipod)s?')],'site_activity':[re.compile('(?mi)(forum|site)\\s(activity|usage)')],'registration_date':[re.compile('(?mi)(registration\\sdate)')],'voter_ids':[re.compile('(?mi)(voter\\sid)s?')],'citizen_status':[re.compile('(?mi)(citizen\\sstatus)')],'contacts':[re.compile('(?mi)(contacts)')],'sexual_orientations':[re.compile('(?mi)(sexual\\sorientation)s?')],'credit_card':[re.compile('(?mi)(credit\\scard)s?')],'spoken_languages':[re.compile('(?mi)(spoken\\slanguage)s?')],'survey_results':[re.compile('(?mi)(survey\\sresults)')],'mesenger':[re.compile('(?mi)(instant\\sidentit(?:y|ies)|instant\\smessenger\\sidentit(?:y|ies)|private\\smessages?)')],'balances':[re.compile('(?mi)(account\\sbalance)s?')],'employers':[re.compile('(?mi)(employers)')],'passport':[re.compile('(?mi)(passport)s?')]}
fields_translation = {'email': 'E-mail', 'username': 'Nome de usuário', 'name': 'Nome', 'password': 'Senha', 'salt': 'Complemento de senha', 'password_hint': 'Dica de senha', '2fa': 'Autenticação de 2 fatores', 'plaintext': 'Texto plano', 'hashed': 'Texto codificado', 'phone': 'Número de telefone', 'ip': 'Endereço IP', 'physical_address': 'Endereço', 'geolocation': 'Geolocalização', 'payment': 'Dados de pagamento', 'gender': 'Gênero', 'dob': 'Data de nascimento', 'device_info': 'Dados de dispositivos', 'site_activity': 'Atividade do site', 'registration_date': 'Data de registro', 'voter_ids': 'Documento de eleitor', 'citizen_status': 'Status de cidadão', 'contacts': 'Contatos', 'sexual_orientations': 'Orientação sexual', 'credit_card': 'Cartão de crédito', 'spoken_languages': 'Idiomas', 'survey_results': 'Resultados de pesquisa', 'mesenger': 'Mensagens', 'balances': 'Balanço das contas', 'employers': 'Empregadores', 'passport': 'Passaporte'}
temp_df = []

def verify_if_identified(row):
  for key in _regex.keys():
    if int(row[key]) != 0:
      return True
  return False


len_identified = len(org_df[org_df.apply(verify_if_identified, axis='columns')])
len_org_df = len(org_df)
for field in _regex:
  row = []
  row.append(fields_translation[field])
  sum = org_df[field].sum()
  row.append(sum)
  row.append(real_br_money_mask(sum)[:-3])
  row.append(real_br_money_mask(sum * 100 / len_identified))
  row.append(real_br_money_mask(sum * 100 / len_org_df))
  row.append(real_br_money_mask(org_df[(org_df[field] == 1) & (org_df['views'] != 0)]['views'].sum())[:-3])
  temp_df.append(row)

df_leaked_data_table = pd.DataFrame(temp_df, columns=['Tipo de informação', 'registro', 'Qtde. registros', 'Identificados (\%)', 'Total (\%)', 'Visualizações'])
df_leaked_data_table.sort_values(by=df_leaked_data_table.columns[1], ascending=False, inplace=True)
df_leaked_data_table.drop(columns=[df_leaked_data_table.columns[1]], inplace=True)
df_leaked_data_table.reset_index(drop=True)
format_table(df_leaked_data_table.head(10).to_latex(
  index=False, 
  caption='Top 10 tipos de informações', 
  label='tab:top10infos', 
  column_format="l|c|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_leaked_data_table.columns.values],
  escape=False
))
#display(df_leaked_data_table)


'\\begin{table}\n\\centering\n\\caption{Top 10 tipos de informações}\n\\label{tab:top10infos}\n\\begin{tabular}{l|c|c|c}\n\\hline\n\\textbf{Tipo de informação} & \\textbf{Qtde. registros} & \\textbf{Identificados (\\%)} & \\textbf{Total (\\%)} & \\textbf{Visualizações} \\\\\n\\hline\n Senha & 868 & 77,71 & 65,91 & 85.373 \\\\\n E-mail & 858 & 76,81 & 65,15 & 84.849 \\\\\n Texto plano & 559 & 50,04 & 42,44 & 56.920 \\\\\n Nome de usuário & 559 & 50,04 & 42,44 & 53.373 \\\\\n Texto codificado & 344 & 30,80 & 26,12 & 32.741 \\\\\n Endereço IP & 309 & 27,66 & 23,46 & 29.622 \\\\\n Nome & 209 & 18,71 & 15,87 & 20.104 \\\\\n Data de nascimento & 182 & 16,29 & 13,82 & 18.311 \\\\\n Número de telefone & 142 & 12,71 & 10,78 & 15.099 \\\\\n Endereço & 139 & 12,44 & 10,55 & 13.751 \\\\\n\\hline\n\\end{tabular}\n\\end{table}\n'

In [ ]:
# Views por dado vazado
_regex={'email':[re.compile('(?mi)(email|e\\s?mail\\saddress|e\\saddress)')],'username':[re.compile('(?mi)(username)')],'name':[re.compile('(?mi)(\\bnames?\\b)')],'password':[re.compile('(?mi)(pass|password)s?')],'salt':[re.compile('(?mi)(salt)s?')],'password_hint':[re.compile('(?mi)(password\\shint|security\\squestions)')],'2fa':[re.compile('(?mi)(2fa)')],'plaintext':[re.compile('(?mi)(plaintext|plain)')],'hashed':[re.compile('(?mi)(hash|md5|sha\\d|bcrypt|ipb)')],'phone':[re.compile('(?mi)((?<!i)phone)')],'ip':[re.compile('(?mi)(\\bip\\b|ipaddress)')],'physical_address':[re.compile('(?mi)(physical|location)s?')],'geolocation':[re.compile('(?mi)(geographic\\slocations)s?')],'payment':[re.compile('(?mi)(payment)\\s(histor(y|ies)|methods?)')],'gender':[re.compile('(?mi)(gender)s?')],'dob':[re.compile('(?mi)(dob|birth)')],'device_info':[re.compile('(?mi)(device\\sinformation|device\\sid|ipad|iphone|ipod)s?')],'site_activity':[re.compile('(?mi)(forum|site)\\s(activity|usage)')],'registration_date':[re.compile('(?mi)(registration\\sdate)')],'voter_ids':[re.compile('(?mi)(voter\\sid)s?')],'citizen_status':[re.compile('(?mi)(citizen\\sstatus)')],'contacts':[re.compile('(?mi)(contacts)')],'sexual_orientations':[re.compile('(?mi)(sexual\\sorientation)s?')],'credit_card':[re.compile('(?mi)(credit\\scard)s?')],'spoken_languages':[re.compile('(?mi)(spoken\\slanguage)s?')],'survey_results':[re.compile('(?mi)(survey\\sresults)')],'mesenger':[re.compile('(?mi)(instant\\sidentit(?:y|ies)|instant\\smessenger\\sidentit(?:y|ies)|private\\smessages?)')],'balances':[re.compile('(?mi)(account\\sbalance)s?')],'employers':[re.compile('(?mi)(employers)')],'passport':[re.compile('(?mi)(passport)s?')]}
temp_df = []
# display(org_df)
for datatype in _regex:
  row = []
  row.append(datatype)
  d = org_df[(org_df[datatype] == 1) & (org_df['views'] != 0)]['views']
  if len(d) == 0:
    continue
  sum = d.sum()
  d = d.describe(percentiles=[.5])
  row.append(int(d['count']))
  row.append(int(d['min']))
  row.append(int(d['50%']))
  row.append(int(d['max']))
  row.append(int(sum))
  temp_df.append(row)  

df_views_per_datatype = pd.DataFrame(temp_df, columns=['Tipo de dados', 'Qtde. anúncios', 'Mínimo', 'Mediana', 'Máximo', 'Soma'])
df_views_per_datatype.sort_values(by=df_views_per_datatype.columns[5], inplace=True, ignore_index=True, ascending=False)
df_views_per_datatype.drop(columns=[df_views_per_datatype.columns[1]], inplace=True)
# format_table(df_views_per_datatype.head(10).to_latex(
#   index=False, 
#   caption='Número de views por anúncio e indústria', 
#   label='tab:top10viewsperdatatype', 
#   column_format="l|c|c|c",
#   float_format="%.2f",
#   header=['\\textbf{%s}' % i for i in df_views_per_datatype.columns.values],
#   escape=False
# ))
display(df_views_per_datatype.head(10))

### Cidadãos

In [ ]:
# Cidadãos
temp_df = []
paises = {'United Kingdom': 'Reino Unido', 'United States': 'Estados Unidos', 'Australia': 'Austrália', 'Brazil': 'Brasil', 'Bulgaria': 'Bulgária', 'Canada': 'Canadá', 'China': 'China', 'Mexico': 'México', 'Philippines': 'Filipinas', 'Russia': 'Rússia', 'Taiwan': 'Taiwan', 'Turkey': 'Turquia'}
for country in citizen_df['country'].unique():
  row = []
  row.append(paises[country])
  d = citizen_df[(citizen_df['country'] == country) & (citizen_df['records'] != 0)]
  if len(d) == 0:
    continue
  row.append(len(d))
  row.append(int(d['records'].describe()['max']))
  temp_df.append(row)  

df_records_by_country = pd.DataFrame(temp_df, columns=['País', 'Qtde. anúncios', 'Max. registros'])
df_records_by_country.sort_values(by=df_records_by_country.columns[1], inplace=True, ignore_index=True, ascending=False)
format_table(df_records_by_country.head(10).to_latex(
  index=False, 
  caption='Número de registros por país', 
  label='tab:top10recordspercountry', 
  column_format="l|c|c",
  float_format="%.2f",
  header=['\\textbf{%s}' % i for i in df_records_by_country.columns.values],
  escape=False
))
#display(df_price_per_idustry.head(10))


In [ ]:
# Cidadãos - Estados EUA
temp_df = []
cidades = ['AL': 'ALABAMA', 'AK': 'ALASKA', 'AS': 'AMERICAN SAMOA', 'AZ': 'ARIZONA', 'AR': 'ARKANSAS', 'CA': 'CALIFORNIA', 'CO': 'COLORADO', 'CT': 'CONNECTICUT', 'DE': 'DELAWARE', 'DC': 'DISTRICT OF COLUMBIA', 'FL': 'FLORIDA', 'GA': 'GEORGIA', 'GU': 'GUAM', 'HI': 'HAWAII', 'ID': 'IDAHO', 'IL': 'ILLINOIS', 'IN': 'INDIANA', 'IA': 'IOWA', 'KS': 'KANSAS', 'KY': 'KENTUCKY', 'LA': 'LOUISIANA', 'ME': 'MAINE', 'MD': 'MARYLAND', 'MA': 'MASSACHUSETTS', 'MI': 'MICHIGAN', 'MN': 'MINNESOTA', 'MS': 'MISSISSIPPI', 'MO': 'MISSOURI', 'MT': 'MONTANA', 'NE': 'NEBRASKA', 'NV': 'NEVADA', 'NH': 'NEW HAMPSHIRE', 'NJ': 'NEW JERSEY', 'NM': 'NEW MEXICO', 'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA', 'ND': 'NORTH DAKOTA', 'MP': 'NORTHERN MARIANA IS', 'OH': 'OHIO', 'OK': 'OKLAHOMA', 'OR': 'OREGON', 'PA': 'PENNSYLVANIA', 'PR': 'PUERTO RICO', 'RI': 'RHODE ISLAND', 'SC': 'SOUTH CAROLINA', 'SD': 'SOUTH DAKOTA', 'TN': 'TENNESSEE', 'TX': 'TEXAS', 'UT': 'UTAH', 'VT': 'VERMONT', 'VA': 'VIRGINIA', 'VI': 'VIRGIN ISLANDS', 'WA': 'WASHINGTON', 'WV': 'WEST VIRGINIA', 'WI': 'WISCONSIN',' 'WY': 'WYOMING']

for state in citizen_df[citizen_df['country'] == 'United States']['state'].unique():
  row = []
  row.append(state)
  d = citizen_df[(citizen_df['country'] == 'United States') & (citizen_df['state'] == state)]
  # & (citizen_df['records'] != 0)
  if len(d) == 0:
    continue
  row.append(len(d))
  row.append(int(d['records'].describe()['max']))
  temp_df.append(row)  

df_records_by_country = pd.DataFrame(temp_df, columns=['Estado', 'Qtde. anúncios', 'Max. registros'])
df_records_by_country.sort_values(by=df_records_by_country.columns[2], inplace=True, ignore_index=True, ascending=False)
display
display(df_records_by_country)


### Escrita da planilha e close do handler

In [ ]:
writer.save()
writer.close()